**OBJECTIVE**
- Implements the feedback system i.e. when a user faves or hides a document for a given MDB query result, this notebook updates the query vector with the document that was faved/hidden and re-ranks the posts already displayed.

In [ ]:
import pickle, tqdm, time, os, tensorflow_hub as hub, numpy as np, pandas as pd
from pymongo import MongoClient
from pprint import pprint

In [ ]:
def getMongoUri(auth_filename='auths/mongo_uri.pkl'):
    with open(auth_filename, 'rb') as file:
        pwd = pickle.load(file)
    return pwd['mongo_uri']

def createMongoClient(db='aita'):
    mongo_uri = getMongoUri()
    client = MongoClient(mongo_uri)
    db = client.aita
    return client, db

In [ ]:
client, db = createMongoClient()

In [ ]:
query_string = 'password' # assume the current query is password
post_id = 'aeeomg' # the id of the post that was hidden/faved
status = 1 # this one was faved(1). hidden = -1.

In [ ]:
# loads USE model O(n) version
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [ ]:
qvector = embed([query_string]).numpy() # convert query to vector

In [ ]:
# get current posts displayed for current query_string
rids = db.queries.find_one({"query": query_string}, projection={'reddit_ids':1})['reddit_ids']
# get the post that was hidden/faved
feedbackPost = db.clean.posts.find_one({"id": post_id}, projection={'vector':1})

In [ ]:
feedbackVector = np.array(feedbackPost['vector']) # get embedding of that post

In [ ]:
SENSITIVITY = 0.75
def update_embedding(q_vector, feedback_vector, feedback):
    new_q = (1 - feedback*SENSITIVITY)*q_vector + feedback*SENSITIVITY*feedback_vector
    return new_q

In [ ]:
qprime = update_embedding(qvector, feedbackVector, status) # move query towards/away from faved/hidden post

In [ ]:
postSubset = []
for p in db.clean.posts.find({'id': {'$in':rids}}, projection={'id':1, 'vector':1}):
    postSubset.append(p)

In [ ]:
vectors = np.array([x['vector'] for x in postSubset])

In [ ]:
vectors.shape

In [ ]:
scores = qprime.dot(vectors.T).flatten()

In [ ]:
ret = []
for i in range(len(postSubset)):
    id_ = postSubset[i]['id']
    score = scores[i]
    ret.append((id_,score))

In [ ]:
ret.sort(key=lambda x:x[1], reverse=True)

In [ ]:
db.queries.update_one({'query':query_string}, {'$set': { "ranked_post_ids" : ret}});

In [ ]:
ret